### Statistical Methods to...
#### assess the magnitude of the forced anthropogenic “signal” relative to the “noise” of internal climate variability to provide a probabilistic projection of climate change impacts in a system with internal variability.

1. quantify change over time:
    * epoch diﬀerence calculated from two 30 year periods
    
    
2. forced signal: the component of change shared by all ensemble members due to external anthropogenic forcing
    * ensemble mean epoch diﬀerence ($X$)
    
    
3. noise: the component of change due to internal climate variability
    * standard deviation of individual epoch diﬀerences around that mean ($\sigma$)
    
    
4. signal-to-noise ratio (SNR) = $\frac{X}{\sigma} \geq 2$

5. Time of Emergence (ToE) analysis: quantifies the speciﬁc year in which a statistically signiﬁcant forced signal is expected to emerge for a given time and region
    * McKinley et al. [2016]: compute linear regressions on <variable>
    for each month and region in each simulation for cumulative 5 year
    intervals (i.e., 1920–1925, 1920–1930, 1920–1935, …, 1920–2100)

    * ToE is the last year of the interval in which the SNR exceeded a
    value of 2 for that interval and all subsequent intervals
    
6. Detection of the forced signal for each month and region ($N_{min}$): approximating the minimum number of simulations required to reach a 95% conﬁdence level
\begin{align}
N_{min} = \frac{8}{\frac{X}{\sigma}^2} \\
\end{align}  
    * If $N_{min}$ is larger than 40 (the number of simulations available
    in the CESM-LE), it is excluded the analysis.

*these methods are from Brady et al 2017 -- below is the `matlab` code used in the methods*
___

## What this would mean is....

Start on one grid cell or area average first. 
Calculate the 1920-1925 trend, then 1920-1930 trend, then 1920-1935 trend and so on for every member. 

    * creating an array sized (E, N) where E is the number of ensemble members and N is the number of 5-year windows. 
    
    * The value would be the slope of the linear trend. 
    
Take the mean over the E dimension and the std deviation over the E dimension. 

Divide the mean array (length N) by the std array (length N). 

Find the point at which that array reaches 2 and does not dip back below it.


`% Use this script to calculate time of emergence for the given variable.
% Will try a few different methods.
VAR_STRUCT = WVELday;
base_year = 1925;
base_index = (base_year - 1920) + 1;
clear database;
for rr = 1:length(regs) % Done for each region.
  disp(['Working on region ' regs{rr} '...']);
  for mm = 1:12
    disp(['Working on month ' int2str(mm) '...']);
    counter = 1;
    current_year = base_year + 5; % First linear regression (five year progressions).
    while current_year <= 2100
      current_index = (current_year - 1920) + 1;
      current_ts = squeeze(VAR_STRUCT.(regs{rr})(mm, base_index:current_index, :));
      ts_length = (current_index - base_index) + 1; % Length of linear regression.
      xx = 1:1:ts_length; % 1-year timesteps.
      % Regress over each member.
      for ss = 1:length(sims)
        temp_ts = current_ts(:,ss);
        temp_p = polyfit(xx, temp_ts', 1);
        current_trend(ss) = temp_p(1);
      end
      temp_sig = mean(current_trend);
      temp_noise = std(current_trend);
      temp_snr = temp_sig / temp_noise;
      database.(regs{rr}).sig(mm, counter) = temp_sig;
      database.(regs{rr}).noise(mm, counter) = temp_noise;
      database.(regs{rr}).snr(mm, counter) = temp_snr;
      counter = counter + 1;
      current_year = current_year + 5; % Move forward five more years.
    end
  end
end`

`% Determine which year = ToE.
yrs = [1930:5:2100];
for rr = 1:length(regs)
  for mm = 1:12
    temp_snr = database.(regs{rr}).snr(mm,:)'; % List of snr time-series.
    temp_snr = abs(temp_snr); % Absolute value to ignore the sign.
    sig_indices = find(temp_snr >= 2); % List of points of time series that exceed two.
    if isempty(sig_indices)
      ToE.(regs{rr})(mm,1) = NaN;
    else
      found = 0;
      while found==0
        checker = [sig_indices(1):1:length(yrs)]'; % Create ideal list if every value is greater than 2 from there out
        if isequalwithequalnans(sig_indices, checker) % Is every value from there on greater than 2?
          ToE_index = sig_indices(1);
          found = 1;
        else
          sig_indices(1) = []; % Remove first value and check again
        end
      end
      ToE.(regs{rr})(mm,1) = yrs(ToE_index);
    end
  end
end`